In [1]:
import getpass
import pandas as pd
import numpy as np
import urllib.parse
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Markdown as md

from shapely import wkb
from scipy.spatial import distance_matrix
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, value
import cvxpy as cp

pd.set_option('display.max_columns', None)

In [2]:
edm_address = getpass.getpass(prompt='EDM server address: ')

print('\nEDM login information')
edm_name = getpass.getpass(prompt='Username: ')
edm_password = getpass.getpass(prompt='Password: ')
edm_password = urllib.parse.quote(edm_password)

%load_ext sql
%sql postgresql://$edm_name:$edm_password@$edm_address/edm
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

# Delete the credential variables for security purpose.
del edm_name, edm_password

EDM server address:  ········



EDM login information


Username:  ········
Password:  ········


In [3]:
# User input for the grid.
grid_id = input('Enter grid ID: ') # awefice

Enter grid ID:  awefice


In [6]:
result = %sql SELECT grid_element_id, \
                        meta, \
                        is_producer, \
                        geometry, \
                        phases\
                FROM grid_element\
                WHERE grid_id = '{grid_id}'\
                    AND type = 'Transformer';

# Convert the results to a data frame.
df_transformers = result.DataFrame()

# Pull out the information from `meta` column saved as JSONB.
df_transformers = pd.concat([df_transformers.drop(['meta'], axis=1),
                             df_transformers['meta'].apply(pd.Series)], axis=1)

# Choose the relevant columns to display transformers. 
df_transformers = df_transformers[['grid_element_id', 'ownership', 'rating_kva', 
                                   'phases', 'voltage_level', 'commission_date', 
                                   'primary_voltage', 'secondary_voltage','is_producer','geometry']]

# Display the results.
df_transformers

Transformers_info = df_transformers[['grid_element_id','is_producer', 'geometry']].set_index('grid_element_id')
Transformers_info['geometry'] = Transformers_info['geometry'].apply(lambda x: wkb.loads(x, hex=True)) 
Transformers_info['x'] = Transformers_info['geometry'].apply(lambda geom: geom.x)
Transformers_info['y'] = Transformers_info['geometry'].apply(lambda geom: geom.y)
Transformers_info = Transformers_info.drop(['geometry'], axis=1)
Transformers_info

,is_producer,x,y
grid_element_id,,,
transformer_16,False,-123.105657,49.267374
transformer_2,True,-123.107138,49.267564
transformer_26,False,-123.103674,49.267342
transformer_31,False,-123.105698,49.266959
transformer_36,False,-123.103687,49.266931
transformer_43,False,-123.101561,49.267313
transformer_47,False,-123.102840,49.266728
transformer_6,True,-123.107136,49.267694
transformer_63,False,-123.104694,49.268323


In [8]:
Ts_Producer = Transformers_info[Transformers_info['is_producer'] == True].index.tolist()
Ts_Producer

['transformer_2', 'transformer_6']

In [9]:
Ts_Consumer = Transformers_info[Transformers_info['is_producer'] == False].index.tolist()
Ts_Consumer

['transformer_16',
 'transformer_26',
 'transformer_31',
 'transformer_36',
 'transformer_43',
 'transformer_47',
 'transformer_63',
 'transformer_92']

In [10]:
Exp_Load_Producer=pd.DataFrame(columns=["transformer_id", "Load"])
for index in Ts_Producer:
    grid_element_id = index
    result = %sql SELECT ge.grid_element_id as transformer_id, \
                            tdss_c.timestamp at time zone 'America/Vancouver' as timestamp, \
                            SUM(tdss_c.value - COALESCE(tdss_p.value, 0)) as "total_kW", ge.meta \
                    FROM grid_element ge \
                    JOIN grid_get_downstream('{grid_id}', ge.grid_element_id, 'false') ggd\
                        ON ggd.grid_id = ge.grid_id \
                    JOIN grid_element_data_source geds_c \
                        ON geds_c.grid_element_id = ggd.grid_element_id \
                        AND geds_c.type = 'CONSUMER' \
                    JOIN ts_data_source_select(geds_c.grid_element_data_source_id, 'kWh') tdss_c \
                        ON true \
                    LEFT JOIN grid_element_data_source geds_p \
                        ON geds_p.grid_element_id = geds_c.grid_element_id \
                        AND geds_p.type = 'PRODUCER' \
                    LEFT JOIN ts_data_source_select(geds_p.grid_element_data_source_id, 'kWh') tdss_p \
                        ON tdss_p.timestamp = tdss_c.timestamp \
                    WHERE ge.grid_element_id = '{grid_element_id}' \
                        AND ggd.type = 'Meter' \
                    GROUP BY ge.grid_element_id, tdss_c.timestamp, ge.meta \
                    ORDER by 2;
    
    # Convert the results to a data frame.
    df_transformer_load = result.DataFrame()
    
    # Pull out the information from the `meta` column saved as JSONB.
    df_transformer_load = pd.concat([df_transformer_load.drop(['meta'], axis=1),
                                    df_transformer_load['meta'].apply(pd.Series)], axis=1)
    
    # Choose the relevant columns to display. 
    df_transformer_load = df_transformer_load[['transformer_id', 'total_kw']]
    
    # Expected load + 1.28 times sd (80% of cases) added to df
    
    new_input = pd.DataFrame([{
        "transformer_id": index,
        "Load": df_transformer_load['total_kw'].mean() + 1.28 * np.std(df_transformer_load['total_kw'])
    }])
    
    Exp_Load_Producer = pd.concat([Exp_Load_Producer, new_input], ignore_index=True)

Exp_Load_Producer

C:\Users\puerc\AppData\Local\Temp\ipykernel_39476\1887534556.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Exp_Load_Producer = pd.concat([Exp_Load_Producer, new_input], ignore_index=True)


,transformer_id,Load
0,transformer_2,75.757616
1,transformer_6,43.201233


In [11]:
Exp_Load_Consumer=pd.DataFrame(columns=["transformer_id", "Load"])
for index in Ts_Consumer:
    grid_element_id = index
    result = %sql SELECT ge.grid_element_id as transformer_id, \
                            tdss_c.timestamp at time zone 'America/Vancouver' as timestamp, \
                            SUM(tdss_c.value - COALESCE(tdss_p.value, 0)) as "total_kW", ge.meta \
                    FROM grid_element ge \
                    JOIN grid_get_downstream('{grid_id}', ge.grid_element_id, 'false') ggd\
                        ON ggd.grid_id = ge.grid_id \
                    JOIN grid_element_data_source geds_c \
                        ON geds_c.grid_element_id = ggd.grid_element_id \
                        AND geds_c.type = 'CONSUMER' \
                    JOIN ts_data_source_select(geds_c.grid_element_data_source_id, 'kWh') tdss_c \
                        ON true \
                    LEFT JOIN grid_element_data_source geds_p \
                        ON geds_p.grid_element_id = geds_c.grid_element_id \
                        AND geds_p.type = 'PRODUCER' \
                    LEFT JOIN ts_data_source_select(geds_p.grid_element_data_source_id, 'kWh') tdss_p \
                        ON tdss_p.timestamp = tdss_c.timestamp \
                    WHERE ge.grid_element_id = '{grid_element_id}' \
                        AND ggd.type = 'Meter' \
                    GROUP BY ge.grid_element_id, tdss_c.timestamp, ge.meta \
                    ORDER by 2;
    
    # Convert the results to a data frame.
    df_transformer_load = result.DataFrame()
    
    # Pull out the information from the `meta` column saved as JSONB.
    df_transformer_load = pd.concat([df_transformer_load.drop(['meta'], axis=1),
                                    df_transformer_load['meta'].apply(pd.Series)], axis=1)
    
    # Choose the relevant columns to display. 
    df_transformer_load = df_transformer_load[['transformer_id', 'total_kw']]
    
    # Expected load + 1.28 times sd (80% of cases) added to df
    
    new_input = pd.DataFrame([{
        "transformer_id": index,
        "Load": df_transformer_load['total_kw'].mean() + 1.28 * np.std(df_transformer_load['total_kw'])
    }])
    
    Exp_Load_Consumer = pd.concat([Exp_Load_Consumer, new_input], ignore_index=True)

Exp_Load_Consumer

C:\Users\puerc\AppData\Local\Temp\ipykernel_39476\2406804298.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Exp_Load_Consumer = pd.concat([Exp_Load_Consumer, new_input], ignore_index=True)


,transformer_id,Load
0,transformer_16,7.540082
1,transformer_26,6.854288
2,transformer_31,7.871367
3,transformer_36,14.515007
4,transformer_43,19.197438
5,transformer_47,33.197110
6,transformer_63,13.216529
7,transformer_92,31.795568


In [13]:
coords_producer = Transformers_info.loc[Transformers_info['is_producer'] == True, ['x', 'y']].to_numpy()
coords_consumer = Transformers_info.loc[Transformers_info['is_producer'] == False, ['x', 'y']].to_numpy()

In [16]:
n_consumers = Exp_Load_Consumer.shape[0]
n_generators = Exp_Load_Producer.shape[0]

cons_coords = coords_consumer
gen_coords = coords_producer

demands = Exp_Load_Consumer['Load'].to_numpy()

distances = np.linalg.norm(cons_coords[:, None, :] - gen_coords[None, :, :], axis=2) # Euclidean distances

x = cp.Variable((n_consumers, n_generators), boolean=True) # Boolean variables
load = cp.sum(cp.multiply(demands[:, None], x), axis=0) # Product of demand by boolean variable
load_avg = cp.sum(load) / n_generators  # Average load attributed to producing generators
load_variance = cp.sum_squares(load - load_avg)  # Variance of load attributed to producing generators
total_distance = cp.sum(cp.multiply(distances, x))   # distance multiplied by boolean variable

alpha = 0.5 # weight of distance
objective = cp.Minimize(alpha * total_distance + (1 - alpha) * load_variance)

constraints = [cp.sum(x[i, :]) == 1 for i in range(n_consumers)]

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS_BB)
print("Estado del problema:", problem.status)

# Resultados
df_result = pd.DataFrame(x.value, index=Ts_Consumer, columns=Ts_Producer)

assignments = np.argwhere(df_result > 0.9)

for i, j in assignments :
    consumidor = df_result.index[i]
    generador = df_result.columns[j]
    print(f" {consumidor} assigned to {generador}")
print("Optimal Loads:", load.value)
print("Total Costo:", problem.value)

Estado del problema: optimal
 transformer_16 assigned to transformer_6
 transformer_26 assigned to transformer_2
 transformer_31 assigned to transformer_2
 transformer_36 assigned to transformer_6
 transformer_43 assigned to transformer_2
 transformer_47 assigned to transformer_2
 transformer_63 assigned to transformer_6
 transformer_92 assigned to transformer_6
Optimal Loads: [67.1202036  67.06718592]
Total Costo: 0.01420682023893314
